In [3]:
from google.cloud import storage

# Configuración
PROJECT_ID = "inventarioproject-471611"
BUCKET_NAME = "proyecto_inventario_bucket"

# Conectar a Google Cloud Storage
client_storage = storage.Client(project=PROJECT_ID)

try:
    bucket = client_storage.bucket(BUCKET_NAME)
    print(f"✅ Conexión exitosa al bucket: {BUCKET_NAME}")

    # Listar archivos
    blobs = list(bucket.list_blobs())
    if not blobs:
        print("⚠️ El bucket está vacío.")
    else:
        print("📂 Archivos encontrados en el bucket:")
        for blob in blobs:
            print("-", blob.name)
except Exception as e:
    print("❌ Error al conectar o listar archivos:")
    print(e)


✅ Conexión exitosa al bucket: proyecto_inventario_bucket
📂 Archivos encontrados en el bucket:
- PurchasesFINAL12312016.csv
- SalesFINAL12312016.csv


In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\Usuario\AppData\Roaming\gcloud\application_default_credentials.json"

In [2]:
import sys
print(sys.executable)



c:\Users\Usuario\OneDrive\Escritorio\InventarioProject\PF-Inventory\venv_inventario\Scripts\python.exe


In [1]:
# ==========================================
# ETL seguro: Verificar bucket, descargar y limpiar CSV
# ==========================================

import os
from google.cloud import storage
import pandas as pd

# ---- Configuración ----
PROJECT_ID = "inventarioproject-471611"           # Tu proyecto de GCP
BUCKET_NAME = "proyecto_inventario_bucket"       # Tu bucket
CSV_FILE_NAME = "PurchasesFINAL12312016.csv"     # Archivo que querés procesar
LOCAL_FILE = "PurchasesFINAL12312016.csv"        # Nombre temporal local

# ---- Configurar credenciales ----
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\Usuario\AppData\Roaming\gcloud\application_default_credentials.json"

# ---- Conectar con GCS ----
try:
    client_storage = storage.Client(project=PROJECT_ID)
    bucket = client_storage.get_bucket(BUCKET_NAME)
    print(f"✅ Conexión exitosa al bucket: {BUCKET_NAME}")
except Exception as e:
    print("❌ No se pudo conectar al bucket. Error:")
    print(e)

# ---- Listar archivos del bucket ----
try:
    blobs = list(bucket.list_blobs())
    if len(blobs) == 0:
        print("⚠️ El bucket está vacío")
    else:
        print("Archivos encontrados en el bucket:")
        for b in blobs:
            print("-", b.name)
except Exception as e:
    print("❌ No se pudieron listar los archivos. Error:")
    print(e)

# ---- Descargar CSV solo si existe ----
if CSV_FILE_NAME in [b.name for b in blobs]:
    print(f"📥 Descargando archivo: {CSV_FILE_NAME}...")
    blob = bucket.blob(CSV_FILE_NAME)
    blob.download_to_filename(LOCAL_FILE)
    print("✅ Descarga completada")
else:
    print(f"❌ No se encontró el archivo {CSV_FILE_NAME} en el bucket")

# ---- Leer y limpiar CSV ----
try:
    df = pd.read_csv(LOCAL_FILE)
    print(f"✅ CSV cargado. Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
    
    # Limpieza básica
    df.columns = df.columns.str.lower().str.strip()  # Normalizar nombres
    df.drop_duplicates(inplace=True)
    
    # Convertir fechas si existe columna 'fecha'
    if 'fecha' in df.columns:
        df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
    
    # Rellenar nulos en columnas numéricas
    for col in df.select_dtypes(include=['number']):
        df[col].fillna(0, inplace=True)
    
    print("✅ Limpieza básica completada")
except Exception as e:
    print("❌ Error al leer o limpiar CSV:")
    print(e)

# ---- Mostrar primeras filas ----
print("Primeras 5 filas del DataFrame:")
print(df.head())

ModuleNotFoundError: No module named 'google'